# Introduction
This [Jupyter](https://jupyter.org/) notebook can be run using [colab.research.google.com](https://colab.research.google.com). See [here](https://colab.research.google.com/notebooks/intro.ipynb) for an intro. Alternatively this can be downloaded and run locally with [Anaconda](https://docs.anaconda.com/anaconda/navigator/). Jupyter and Anaconda should be installed in all AUT engineering and computer science labs. I recommend using the Goolge web-interface.

The benefit of using Jupyter is that code snippets can be run live (Python is running in the background).

The version on Github is static; markdown is rendered but code cannot be executed. All code can be copied and pasted into your favourite text editor or IDE and *should* run with Python 3.x ;)

You are encouraged to use any programming language you feel comfortable with, this is simply an example using Python (and Jupyter is designed for Python demonstrations).

---

# Tutorial: Proof of Work - Mining 

**_Proof of work_** is the unique contribution to the bitcoin protocol that achieves two aims:
1. allows nodes to vote with their hashpower to achieve consensus
2. keeps the network secure by aligning incentives.

We have seen in the previous tutorials some of the uses of hashing with the `hashlib` library. Because a standard hash function like `sha256` is random, meaning there is no way to game the system or shortcut the output, it allows equal participation from anyone in the network. If one participant chooses to dedicate all of their computers and resources to hashing, then they are allowed more "votes" in the network and will receive proportionally more rewards for their effort.

This whole process is referred to as _mining_ because it is analagous to gold mining.

So lets get mining!

### Prepare a block to be mined
Press `shift-enter` in the cell to run it. You may have to wait for the environment to initialize if this is the first time. There is a status bar above.

The out put will appear directly below the code block.

In [ ]:
# we will need pickle to turn our block into a byte object
# sha256 operates one byte at a time
import hashlib
import pickle

# create a new block using sample values
block = {   'timestamp': 1553200425.597771,
            'transactions': ['a','b','c'],
            'prev_hash': '00574faf29dbb37a3b12c2f8f5c05cf03c708313d4ad1d5d968cd2438beea104',
            'merkle_root': 'aa0522a507b6499cb3512494908ac9220',
        }

# pickle the block
pickled_block = pickle.dumps(block)
print(pickled_block)

This block is in byte form (the `'b`) and can now be hashed. Lets see what it looks like.

In [ ]:
# hash the block and output a digest of hexidecimal digits
hashed_block = hashlib.sha256(pickled_block).hexdigest()
print(hashed_block)

Every time we hash this same block with sha256 we will get the same unique output. In this case repeated hashing will not change the output. 

Lets introduce a variable that is added to the block. This variable doesn't change the data in the block, but allows for a different hash output. It is a **number** we will use **once**, then hash, test, update, and hash again. This is called a **nonce**.

This simple example shows how a nonce changes the hash:

In [ ]:
# choose 16 because a hex output has 16 digits
for nonce in range(16):
    print(str(nonce)+'_block_nonce')

Now output the hash of the above strings:

In [ ]:
for nonce in range(16):  
    print(hashlib.sha256((str(nonce)+'block_nonce').encode()).hexdigest())

There should be no pattern to the hashed values above. All that was changed was the nonce. Since a hexidecial number has 16 possible digits (0-9,a-f), there should be a one-sixteenth probability that the first digit of the digest is a `0`. (How many are above?)

What is the probability that the first *two* digits are: `00`? 

$P($ `00` $)=\left(\frac{1}{16}\right)\left(\frac{1}{16}\right)=\frac{1}{16^2}=\frac{1}{256}=0.3906\%$

So the probability the first $n$ digits are `0` is:

$P($ `0` $^n)=\frac{1}{16^n}$

As the number of zeroes increase, the probability of finding a hash with those leading zeroes goes down. If we are looking for a hash with `00` at the beginning this is called the _target difficulty_. Mining a successful block means finding a hash that is below the target value. One way to do this is to take the hash string and compare the first digit(s).

In [ ]:
# set the number of leading zeroes
target = 1

# loop through the hashes checking each one
for nonce in range(16):  
    hash = hashlib.sha256((str(nonce)+'block_nonce').encode()).hexdigest()
    print(str(hash))
    
    # check the string up to the indexed difficulty and compare
    # note python string manipulation: '0'*5 = 00000; this is string multiplication
    if hash[:target] == '0'*target:
        print('Block Found!')
        break
        
    # end of range condition
    if nonce == 15: print('Block not found')
    

Our loop breaks after 12 iterations because we've satisfied the condition; in this case find a single leading zero.

***

Up to this point we have just been hashing a string `'_block_nonce'`. Lets now use our block.

In [ ]:
# !!caution!! - setting difficulty too high could result in waiting until the heat death of the universe to find your block
difficulty = 4
nonce = 1
block_found = False

# this will loop until a block is found
while block_found == False:
    # add a nonce value to the block, here it is prepended
    work = hashlib.sha256(bytes(nonce) + pickled_block)
    
    # check to see if the hash meets the target
    if work.hexdigest()[:target] == '0'*target:
        block_found = True
        
        # output some data
        print('Target Difficulty: ' + str(target))
        print('Nonce: ' + str(nonce))
        print('Proof-of-work: ' + work.hexdigest())
        
    # update the nonce before looping
    # this ensures the block is different before it is hashed again    
    nonce += 1
    
# you may have to wait 10-20 seconds for output to appear below. Change the difficulty value and compare.

The nonce indicates how many times the loop was run which is how many `SHA-256` hashes were calculated. When you hear people say that bitcoin mining involves "solving complex puzzles" this is what they are referring to. Sidebar: these **aren't puzzles at all!** It is simply an algorithm to manipulate bits and compare to a target condition.

```python
Difficulty: 4
Nonce: 102591
Proof-of-work: 0000ee2830b3999eec469cc16a81f0eb6a163e84ece694803adaf3392b74804b
```

Above I set the difficulty to 4 and can see that the hash was calculated 102,591 times before meeting the condition.

### Add a system timer for analysis

Change the difficulty value above and compare the different outputs. Adding a timer can help with the comparison.

In [ ]:
target = 4
nonce = 1
block_found = False

# access the system clock and record the start time
import time
start = time.time()
while block_found == False:
    work = hashlib.sha256(bytes(nonce) + pickled_block)
    if work.hexdigest()[:target] == '0'*target:
        end = time.time()
        block_found = True
        print('Target Difficulty: ' + str(target))
        print('Nonce: ' + str(nonce))
        print('Proof-of-work: ' + work.hexdigest())
        
        # calculate and print the time; this will output in seconds
        print('Elapsed time: ' + str(end - start))
    nonce += 1

### Generate random blocks

Re-running this will take similar times because we keep hashing the same block with the same nonce. In practice, once a block is found, the miners start mining the next block which contains different transactions. Introducing some random bytes into our block can help you with generating "new" blocks to analyse mining and difficulty performance.

In [ ]:
# secrets provides access to a secure random number generator. 
# Refer to documentation for what this means. 
# Always be cautious when generating random data for important purposes like keys.
import secrets
# generate random data to append to a block
random_data = secrets.token_bytes(4)

target = 4
nonce = 1
block_found = False
start = time.time()
while block_found == False:
    # the block now contains 4 random bytes of data
    work = hashlib.sha256(bytes(nonce) + pickled_block + random_data)
    if work.hexdigest()[:target] == '0'*target:
        end = time.time()
        block_found = True
        print('Target Difficulty: '+ str(target))
        print('Nonce: ' + str(nonce))
        print('Proof-of-work: ' + work.hexdigest())
        print('Elapsed time: ' + str(end - start))
    nonce += 1

These results should be fresh. Every time you run this block or access `secrets.token_bytes(4)` the block will change.

Compare results I just got:
```
Target Difficulty: 4
Nonce: 276851
Proof-of-work: 000067ddf3ef7837481dc818093aea15bce9e4da166bdfc02e9916040e53cb28
Elapsed time: 37.96086359024048

Target Difficulty: 4
Nonce: 22947
Proof-of-work: 0000f9b8371941856efe3f94f09b36bbc9580de5451ab058f04c0cb53580154a
Elapsed time: 0.6624038219451904
```

This shows the lottery-like nature of using hash functions and target difficulty values. The second set found a proof-of-work in 0.66 s and the first one took 57 times longer!

---

## Probability
As more miners come online to try to earn coins (and process transactions) the difficulty must adjust to maintain the blocktime. Without this difficulty adjustment the group with the most hashpower would have already mined all the available coins in any proof of work network, likely causing the value to crash.

The difficulty set in this tutorial goes up by powers of 16; there is a sixteen-fold increase in time expected to find a block as the leading zeroes go from `0000` to `00000`. (Try with 5 or 6 and see how long you have to wait.) What if you want difficulty to increase only by a factor or two? (See exercise #1)

Lets look at the probability of finding a block.

In [ ]:
# one leading zero to start
diff = 1
prob = 1/16
print(prob)

In [ ]:
diff = 2
prob = 1/(16**diff)
print(prob)

In [ ]:
#loop through a range and print the results
#range(start,stop,step)
for diff in range(1,16,1):
  prob = 1/(16**diff)
  #formatted to show the relative value
  print("%.20f" % prob)

In [ ]:
#same as above only stored in an array (python list) to plot
prob=[]
for diff in range(1,11,1):
  prob.append(1/(16**diff))
print(prob)


In [ ]:
#plot using matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_xlabel('difficulty')
ax.set_ylabel('probability')
ax.plot(prob,'r*')
plt.show()


This shows the probability for a single hash to reach our target difficulty: a 6.25% chance to find 1-leading zero. 

The proof of work for Bitcoin block 642428 was:
`00000000000000000006a19bf1ef2f829baf3937d3ed11d6d545b947f3214aef`.
This has 19 leading zeroes! To increase your odds you can (a) use a faster processor (b) use more processors in parallel.

## Difficulty
Up to this point we have used the term target difficulty to mean the number of leading bits that are zero in a hexidecimal format. When looking up difficulty on the [web](http://bitcoin.sipa.be/) you will get a different value, for example: the difficulty when the above block was mined was `16847561611550.27`.

What does this mean? Lets use a smaller set for analogy. Suppose the set size was 100; all integers between 0 and 100 are valid possibilities in our lottery. If the target is set at 5, then any number less than 5 is a winner: 0,1,2,3,4. 
Now, the formula for difficulty is:

$\text{difficulty}=\frac{\text{set size}}{\text{target}}$

so, $\text{difficulty}=\frac{100}{5}=20$ attempts to find a number less than 5.

The difficulty is a measure of how many times you need to attempt the lottery to find a winner, as a statistical measure over time.


In [ ]:
#calculate bitcoin's difficulty
#the maximum number of values that can be output 
#from SHA256 are 2^256 -1, or approximately 
#0x00000000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
set_size = 0x00000000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
print(set_size)

In [ ]:
#recently found block's proof of work (copied from web)
recent_PoW = 0x00000000000000000006a19bf1ef2f829baf3937d3ed11d6d545b947f3214aef
print(recent_PoW)

In [ ]:
#calculate difficulty value
difficulty = set_size / recent_PoW
print(difficulty)

# Summary

In this tutorial we have:<br>
 - added a nonce to our block
 - hashed blocks until they meet a difficulty requirement
 - investigated probabilities using `matplotlib`
 - defined and calculated difficulty
 
What we have __not__ done is:<br>
 - set a target value that is not a power of 16
 - dynamically adjusted target values
 - appended mined blocks to a chain
 
Python libraries that this code depends on:
 - [hashlib](https://docs.python.org/3/library/hashlib.html)
 - [pickle](https://docs.python.org/3/library/pickle.html)
 - [secrets](https://docs.python.org/3/library/secrets.html)
 - [time](https://docs.python.org/3/library/time.html)

Useful links:
 - [hex to decimal converter](https://www.rapidtables.com/convert/number/hex-to-decimal.html)
 - [matplotlib quick tutorial](https://www.tutorialspoint.com/matplotlib/matplotlib_quick_guide.htm)
 - [bitcoin difficulty](https://en.bitcoin.it/wiki/Difficulty)

 ---

# Exercises

1. The calculated difficulty (above) was `42446515794025.11`. Compare this to the difficulty of the network of `16847561611550.27`. Why is there this difference between the two? Investigate the block [here](https://blockchair.com/bitcoin/block/642428).
1. Mining difficulty is not only represented by leading zeroes (powers of 16) but by a value (hex number) that is recalculated every 2016 blocks (in Bitcoin). Update your code to include a target difficulty value such that:
```python
if hash < target: block_found = True
```

2. Write a script to modify the difficulty every time a block is found faster than expected (increase difficulty) and readjusts if a block is found slower than expected (decrease difficulty).
3. Calculate the hashpower of your PC and estimate how long it would take to find a block in the [present bitcoin network](http://bitcoin.sipa.be/).